In [1]:
import pandas as pd
import imblearn
import numpy as np
import traceback
pd.set_option('display.max_rows', 20,'max_info_columns', 9999,'display.max_columns', 9999)
import matplotlib.pyplot as plt
import copy
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing 
import datetime as dt
import gc
from sklearn import svm,linear_model
from collections import Counter
from sklearn.model_selection import train_test_split
import csv
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder,Imputer

from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedKFold


from sklearn.feature_selection import RFE 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
import xlrd
warnings.filterwarnings('ignore')

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('/home/tsl-yu/文档/天池-盐城-乘用车零售销量/[new] yancheng_train_20171226.csv')

test=pd.read_csv('/home/tsl-yu/文档/天池-盐城-乘用车零售销量/yancheng_testA_20171225.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20157 entries, 0 to 20156
Data columns (total 32 columns):
sale_date                20157 non-null int64
class_id                 20157 non-null int64
sale_quantity            20157 non-null int64
brand_id                 20157 non-null int64
compartment              20157 non-null int64
type_id                  20157 non-null int64
level_id                 20157 non-null object
department_id            20157 non-null int64
TR                       20157 non-null object
gearbox_type             20157 non-null object
displacement             20157 non-null float64
if_charging              20157 non-null object
price_level              20157 non-null object
price                    20157 non-null object
driven_type_id           20157 non-null int64
fuel_type_id             20157 non-null object
newenergy_type_id        20157 non-null int64
emission_standards_id    20157 non-null int64
if_MPV_id                20157 non-null int64
if_luxur

In [4]:
test

,predict_date,class_id,predict_quantity
0,201711,103507,NaN
1,201711,124140,NaN
2,201711,125403,NaN
3,201711,136916,NaN
4,201711,169673,NaN
5,201711,175962,NaN
6,201711,178529,NaN
7,201711,186250,NaN
8,201711,194201,NaN
9,201711,194450,NaN


In [4]:
train.head()

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,price_level,price,driven_type_id,fuel_type_id,newenergy_type_id,emission_standards_id,if_MPV_id,if_luxurious_id,power,cylinder_number,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,201609,289403,94,12,2,1,1,1,6,MT,1.8,L,8-10W,9.5,1,1,1,1,2,1,98,4,170,4440,1833,1545,1695,1320,5,2700,1556,1562
1,201609,745137,435,637,3,2,1,2,6,DCT,1.6,L,10-15W,13.2,1,1,1,1,2,1,94,4,159,4534,1823,1483,1711,1336,5,2648,1553,1544
2,201609,714860,180,831,3,2,2,3,6,AT,1.8,L,10-15W,14.4,1,1,1,1,2,1,105,4,176,4720,1815,1465,1860,1459,5,2770,1579,1589
3,201609,175962,40,750,3,2,1,4,6,AT,1.6,L,10-15W,8.6,1,1,1,1,2,1,81,4,155,4475,1706,1469,1625,1145,5,2603,1460,1500
4,201609,270690,19,98,2,3,3,1,5,MT,1.5,L,5WL,-,2,1,1,1,1,1,82,4,146.5,4415,1685,1850,1825,1236,5,2720,1420,1440


In [5]:
len(set(train['class_id']))

140

In [6]:
len(set(train['sale_date']))

70

In [7]:
train.shape

(20157, 32)

## 以日期为划分标准，划分数据 

In [8]:
def split_data(df):
    l=[]
    for i,data in df.groupby('sale_date'):
        l.append(data)
        
    return l

## 对象类型处理 

In [47]:
def handle_object(df):
    
    en=LabelEncoder()
    df['level_id']=en.fit_transform(train['level_id'])
    
    df['TR']=df['TR'].replace({'8;7':7.5,'5;4':4.5}).astype(float)
    
    df['gearbox_type']=en.fit_transform(train['gearbox_type'])
    
    df['if_charging']=en.fit_transform(train['if_charging'])
    
    dict_price_l={'20-25W':22.5, '25-35W':30, '15-20W':17.5, '5-8W':6.5,
                  '35-50W':42.5, '8-10W':9, '10-15W':12.5, '50-75W':62.5, '5WL':5}
    df['price_level']=df['price_level'].map(dict_price_l)
    
    
    df['price']=df['price'].replace({'-':None}).astype(float)
    
    df['fuel_type_id']=df['fuel_type_id'].replace({'-':None}).astype(float)
    
    df['power']=df['power'].replace({'-':None,'81/70':81}).astype(float)
    
    df['engine_torque']=df['engine_torque'].replace({'-':None,'155/140':155}).astype(float)
    
    dict_rated_p={'6-7':6.5, '9':9, '5':5, '5-7':6, '4-5':4.5, '4':4, '7-8':7.5, '5-8':6.5, '7':7, '6-8':6.5}
    
    df['rated_passenger']=df['rated_passenger'].map(dict_rated_p)
    
    return df

In [49]:
train1=handle_object(train)

In [6]:
train1.info()

NameError: name 'train1' is not defined

In [7]:
train.rename(columns={'class_id':'c'})

,sale_date,c,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,price_level,price,driven_type_id,fuel_type_id,newenergy_type_id,emission_standards_id,if_MPV_id,if_luxurious_id,power,cylinder_number,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,201609,289403,94,12,2,1,1,1,6,MT,1.8,L,8-10W,9.5,1,1,1,1,2,1,98,4,170,4440,1833,1545,1695,1320,5,2700,1556,1562
1,201609,745137,435,637,3,2,1,2,6,DCT,1.6,L,10-15W,13.2,1,1,1,1,2,1,94,4,159,4534,1823,1483,1711,1336,5,2648,1553,1544
2,201609,714860,180,831,3,2,2,3,6,AT,1.8,L,10-15W,14.4,1,1,1,1,2,1,105,4,176,4720,1815,1465,1860,1459,5,2770,1579,1589
3,201609,175962,40,750,3,2,1,4,6,AT,1.6,L,10-15W,8.6,1,1,1,1,2,1,81,4,155,4475,1706,1469,1625,1145,5,2603,1460,1500
4,201609,270690,19,98,2,3,3,1,5,MT,1.5,L,5WL,-,2,1,1,1,1,1,82,4,146.5,4415,1685,1850,1825,1236,5,2720,1420,1440
5,201609,290854,53,537,2,1,1,5,0,CVT,2.0,L,15-20W,21,1,1,1,1,2,1,114,4,190,4585,1820,1685,2010,1540,5,2620,1580,1580
6,201609,692703,81,807,2,1,1,1,5,MT,1.3,T,10-15W,13.3,1,1,1,1,2,1,101,4,202,4510,1852,1677,1900,1450,5,2650,1575,1570
7,201609,978089,37,12,2,2,1,1,5,MT,1.5,L,5-8W,8.9,1,1,1,1,2,1,80,4,140,4425,1789,1470,1611,1236,5,2650,1502,1492
8,201609,219195,115,638,2,1,3,1,6,AT,1.5,T,10-15W,11.1,1,1,1,1,2,1,110,4,210,4335,1814,1695,1895,1520,5,2560,1525,1520
9,201609,851857,78,75,2,1,2,2,8,AT,2.0,T,25-35W,44.8,3,1,1,1,2,2,193,4,400,4812,1903,1680,2375,1885,5,2857,1645,1645


In [8]:
train

,sale_date,class_id,sale_quantity,brand_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,price_level,price,driven_type_id,fuel_type_id,newenergy_type_id,emission_standards_id,if_MPV_id,if_luxurious_id,power,cylinder_number,engine_torque,car_length,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track
0,201609,289403,94,12,2,1,1,1,6,MT,1.8,L,8-10W,9.5,1,1,1,1,2,1,98,4,170,4440,1833,1545,1695,1320,5,2700,1556,1562
1,201609,745137,435,637,3,2,1,2,6,DCT,1.6,L,10-15W,13.2,1,1,1,1,2,1,94,4,159,4534,1823,1483,1711,1336,5,2648,1553,1544
2,201609,714860,180,831,3,2,2,3,6,AT,1.8,L,10-15W,14.4,1,1,1,1,2,1,105,4,176,4720,1815,1465,1860,1459,5,2770,1579,1589
3,201609,175962,40,750,3,2,1,4,6,AT,1.6,L,10-15W,8.6,1,1,1,1,2,1,81,4,155,4475,1706,1469,1625,1145,5,2603,1460,1500
4,201609,270690,19,98,2,3,3,1,5,MT,1.5,L,5WL,-,2,1,1,1,1,1,82,4,146.5,4415,1685,1850,1825,1236,5,2720,1420,1440
5,201609,290854,53,537,2,1,1,5,0,CVT,2.0,L,15-20W,21,1,1,1,1,2,1,114,4,190,4585,1820,1685,2010,1540,5,2620,1580,1580
6,201609,692703,81,807,2,1,1,1,5,MT,1.3,T,10-15W,13.3,1,1,1,1,2,1,101,4,202,4510,1852,1677,1900,1450,5,2650,1575,1570
7,201609,978089,37,12,2,2,1,1,5,MT,1.5,L,5-8W,8.9,1,1,1,1,2,1,80,4,140,4425,1789,1470,1611,1236,5,2650,1502,1492
8,201609,219195,115,638,2,1,3,1,6,AT,1.5,T,10-15W,11.1,1,1,1,1,2,1,110,4,210,4335,1814,1695,1895,1520,5,2560,1525,1520
9,201609,851857,78,75,2,1,2,2,8,AT,2.0,T,25-35W,44.8,3,1,1,1,2,2,193,4,400,4812,1903,1680,2375,1885,5,2857,1645,1645
